In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
import pyspark

create spark session, not spark context like before

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Python Spark SQL Examples").getOrCreate()

In [5]:
cdf=spark.read.load("customers.txt",format="csv",sep="\t",inferSchema="true",header="true")

In [6]:
cdf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



spark apis infer the datatypes from the inferSchema parameter

In [7]:
cdf.select("customer_name").show() #cdf.select("customer_name") gives an object

+----------------+
|   customer_name|
+----------------+
|     Mary Torres|
|      Jose Haley|
|      Mary Smith|
|  Richard Maddox|
|  Margaret Booth|
|  Mary Henderson|
|     Lisa Walker|
|   Jonathan Hill|
|Carolyn Sheppard|
|    Mary Mendoza|
|   Michael Smith|
|    James Holmes|
|     Mary Dawson|
|    Adam Marquez|
|    Gloria Smith|
|       Mary Webb|
|  Nancy Alvarado|
|  Russell Flores|
|    Denise Smith|
|  Jose Dickerson|
+----------------+
only showing top 20 rows



In [8]:
cdf.select(cdf["customer_id"],cdf["customer_name"]).show()

+-----------+----------------+
|customer_id|   customer_name|
+-----------+----------------+
|      11039|     Mary Torres|
|       5623|      Jose Haley|
|       5829|      Mary Smith|
|       6336|  Richard Maddox|
|       1708|  Margaret Booth|
|      10227|  Mary Henderson|
|        839|     Lisa Walker|
|       7604|   Jonathan Hill|
|       6485|Carolyn Sheppard|
|       4737|    Mary Mendoza|
|       5973|   Michael Smith|
|       9205|    James Holmes|
|        138|     Mary Dawson|
|        371|    Adam Marquez|
|       9285|    Gloria Smith|
|       1209|       Mary Webb|
|       3021|  Nancy Alvarado|
|       3354|  Russell Flores|
|      11684|    Denise Smith|
|      11144|  Jose Dickerson|
+-----------+----------------+
only showing top 20 rows



In [9]:
cdf.filter(cdf['customer_state']=='CA').show()

+-----------+----------------+---------------+--------------+----------------+
|customer_id|   customer_name|  customer_city|customer_state|customer_zipcode|
+-----------+----------------+---------------+--------------+----------------+
|       5577|      Mary Smith|        Modesto|            CA|           95350|
|       1745|      Mary Smith|Rowland Heights|            CA|           91748|
|      11444|Kathleen Patrick|      San Diego|            CA|           92109|
|       8846|    Thomas Smith|          Indio|            CA|           92201|
|       6237|  Bobby Anderson|       El Cajon|            CA|           92020|
|       4085|       Mary Carr|  Panorama City|            CA|           91402|
|       8705|  Patricia Smith|       Stockton|            CA|           95207|
|       3669|       Mary Soto| San Bernardino|            CA|           92410|
|       6101|      Mary Smith|    Los Angeles|            CA|           90033|
|      11697|  Jessica Thomas|  Laguna Niguel|      

how to get a derived dataframe or a temporary view

In [10]:
cdf.createOrReplaceTempView("cust")

In [11]:
query="SELECT * FROM cust"
c=spark.sql(query)

In [12]:
c.show()

+-----------+----------------+-------------+--------------+----------------+
|customer_id|   customer_name|customer_city|customer_state|customer_zipcode|
+-----------+----------------+-------------+--------------+----------------+
|      11039|     Mary Torres|       Caguas|            PR|             725|
|       5623|      Jose Haley|     Columbus|            OH|           43207|
|       5829|      Mary Smith|      Houston|            TX|           77015|
|       6336|  Richard Maddox|       Caguas|            PR|             725|
|       1708|  Margaret Booth|    Arlington|            TX|           76010|
|      10227|  Mary Henderson|       Caguas|            PR|             725|
|        839|     Lisa Walker|       Caguas|            PR|             725|
|       7604|   Jonathan Hill|      Phoenix|            AZ|           85040|
|       6485|Carolyn Sheppard|Pompano Beach|            FL|           33063|
|       4737|    Mary Mendoza|       Caguas|            PR|             725|

In [13]:
type(c)

pyspark.sql.dataframe.DataFrame

In [14]:
q1="SELECT customer_state, COUNT(*) AS state_count FROM cust GROUP BY customer_state HAVING state_count>=50"

In [15]:
c50=spark.sql(q1)

In [16]:
c50.show()

+--------------+-----------+
|customer_state|state_count|
+--------------+-----------+
|            CA|        187|
|            NY|         79|
|            TX|         62|
|            PR|        505|
+--------------+-----------+



In [17]:
c50.write.json("write.json")

write.json has so many parts, its difficult to handle


In [18]:
c50.coalesce(1).write.json("c50.json")